In [5]:
pip install boto3


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import boto3
import os
# Create S3 client
s3 = boto3.client('s3')

# Bucket name
bucket_name = 'bolzhau2'

# Local directory path
local_directory = r'C:\Users\ww\Desktop\данные'

# Upload all files in the directory and its subdirectories
for root, dirs, files in os.walk(local_directory):
    for file_name in files:
        # Construct the full local path of the file
        local_file_path = os.path.join(root, file_name)
        
        # Construct the S3 object key (path in the bucket)
        relative_path = os.path.relpath(local_file_path, local_directory)
        s3_key = f"data/{relative_path}"

        # Upload the file to S3
        s3.upload_file(local_file_path, bucket_name, s3_key)

        print(f"Uploaded {local_file_path} to s3://{bucket_name}/{s3_key}") 

print("All files uploaded successfully!")

In [6]:
from datasets import load_dataset

ds = load_dataset("kz-transformers/multidomain-kazakh-dataset")

ModuleNotFoundError: No module named 'datasets'

In [3]:
import tensorflow as tf

import numpy as np
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
import chardet

In [7]:
# Read, then decode for py2 compat.
with open('kk.txt', 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']
    
text = open('kk.txt', 'rb').read().decode(encoding)

# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 207077 characters


In [8]:
# Take a look at the first 250 characters in text
print(text[:207077])

Kitaptardy tabyńyz.
Qazir siz oqyńyz.
Shulyǵyńyzdy sheshińiz.
Biz etti jedik.
Qyzymdy kórdińiz be?
Qyzymdy kórdiń be?
Láıli barlyq aqshany qalady.
Barlyq aqshany qalamadym.
Antısıonızm - antısemıtızm emes.
Mennad jumysta edi.
Sýshıdi jaqsy kórmeımin.
Sýshı jediń be?
Sýshıdi jaqsy kóresiń be?
Mundaı ázilderdi jaqsy kórmeımin.
Men irimshikti jaqsy kóremin.
Patshaıymdy kórdim.
Ol esikti japty.
Myna áıeldiń aty kim?
Qyzyńdy maǵan ber.
Anamdy óltirdiń.
Avtovokzal qaıda?
Kitap oqymadyń ba?
Men seni súıemin.
Men seni jaqsy kórem.
Dúısenbini jek kóremin.
Men seni jek kóremin.
Sýshıdi jek kóremin.
Ony jek kórmeımin, biraq jaqsy da kórmeımin.
Ol dosym emes.
Oqýǵa ýaqytym joq.
Sen et jeısiń.
Siz et jeısiz.
Sen et jep otyrsyń.
Siz et jep otyrsyz.
Nensı apama uqsaıdy.
Eshkimge meniń munda bolǵanymdy aıtpa.
Ár oqýshynyń óz kompıýteri bar.
Men ıtterdi jaqsy kóremin.
Oǵan ne aıttyń?
Tomdy kúttirdim.
Tom da erte turdy edi.
Qaryndashymdy túsirip aldym.
Polısıa qyzmetkerlerine kezekte alkogoldik ishimdik

In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

176 unique characters


Vectorize the text
Before training, you need to convert the strings to a numerical representation.

The tf.keras.layers.StringLookup layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [10]:
example_texts = ['ләйлік', 'қал']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'\xd0\xbb', b'\xd3\x99', b'\xd0\xb9', b'\xd0\xbb', b'\xd1\x96',
  b'\xd0\xba']                                                    ,
 [b'\xd2\x9b', b'\xd0\xb0', b'\xd0\xbb']]>

Now create the tf.keras.layers.StringLookup layer:

In [11]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [12]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[132, 167, 130, 132, 154, 131], [158, 121, 132]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover
human-readable strings from it. For this you can use tf.keras.layers.StringLookup(..., invert=True).

Note: Here instead of passing the original vocabulary generated with sorted(set(text)) use the get_vocabulary() method 
of the tf.keras.layers.StringLookup layer so that the [UNK] tokens is set the same way.

In [13]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a tf.RaggedTensor of characters:

In [14]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'\xd0\xbb', b'\xd3\x99', b'\xd0\xb9', b'\xd0\xbb', b'\xd1\x96',
  b'\xd0\xba']                                                    ,
 [b'\xd2\x9b', b'\xd0\xb0', b'\xd0\xbb']]>

In [15]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'\xd0\xbb\xd3\x99\xd0\xb9\xd0\xbb\xd1\x96\xd0\xba',
       b'\xd2\x9b\xd0\xb0\xd0\xbb'], dtype=object)

In [16]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

The prediction task
Given a character, or a sequence of characters, what is the most probable next character? 
This is the task you're training the model to perform. The input to the model will be a sequence 
of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the 
characters computed until this moment, what is the next character?

Create training examples and targets
Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [19]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(207077,), dtype=int64, numpy=array([ 36,  57,  68, ..., 134,  13,   1], dtype=int64)>

In [20]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [21]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode(encoding))

K
i
t
a
p
t
a
r
d
y


In [22]:
seq_length = 100

In [23]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq)) 

tf.Tensor(
[b'K' b'i' b't' b'a' b'p' b't' b'a' b'r' b'd' b'y' b' ' b't' b'a' b'b'
 b'y' b'\xc5\x84' b'y' b'z' b'.' b'\n' b'Q' b'a' b'z' b'i' b'r' b' ' b's'
 b'i' b'z' b' ' b'o' b'q' b'y' b'\xc5\x84' b'y' b'z' b'.' b'\n' b'S' b'h'
 b'u' b'l' b'y' b'\xc7\xb5' b'y' b'\xc5\x84' b'y' b'z' b'd' b'y' b' ' b's'
 b'h' b'e' b's' b'h' b'i' b'\xc5\x84' b'i' b'z' b'.' b'\n' b'B' b'i' b'z'
 b' ' b'e' b't' b't' b'i' b' ' b'j' b'e' b'd' b'i' b'k' b'.' b'\n' b'Q'
 b'y' b'z' b'y' b'm' b'd' b'y' b' ' b'k' b'\xc3\xb3' b'r' b'd' b'i'
 b'\xc5\x84' b'i' b'z' b' ' b'b' b'e' b'?' b'\n' b'Q' b'y'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [24]:
for seq in sequences.take(5):
    # Convert the sequence of IDs to characters using text_from_ids
    decoded_text = text_from_ids(seq)
    
    # Convert the TensorFlow tensor to a numpy array and decode it to a string
    decoded_string = tf.strings.reduce_join(decoded_text).numpy().decode('utf-8')
    
    # Print the resulting string
    print(decoded_string)

Kitaptardy tabyńyz.
Qazir siz oqyńyz.
Shulyǵyńyzdy sheshińiz.
Biz etti jedik.
Qyzymdy kórdińiz be?
Qy
zymdy kórdiń be?
Láıli barlyq aqshany qalady.
Barlyq aqshany qalamadym.
Antısıonızm - antısemıtızm em
es.
Mennad jumysta edi.
Sýshıdi jaqsy kórmeımin.
Sýshı jediń be?
Sýshıdi jaqsy kóresiń be?
Mundaı ázi
lderdi jaqsy kórmeımin.
Men irimshikti jaqsy kóremin.
Patshaıymdy kórdim.
Ol esikti japty.
Myna áıeld
iń aty kim?
Qyzyńdy maǵan ber.
Anamdy óltirdiń.
Avtovokzal qaıda?
Kitap oqymadyń ba?
Men seni súıemin


For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:




In [25]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [26]:
split_input_target(list("қаракөз"))

(['қ', 'а', 'р', 'а', 'к', 'ө'], ['а', 'р', 'а', 'к', 'ө', 'з'])

In [27]:
dataset = sequences.map(split_input_target)

In [28]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Kitaptardy taby\xc5\x84yz.\nQazir siz oqy\xc5\x84yz.\nShuly\xc7\xb5y\xc5\x84yzdy sheshi\xc5\x84iz.\nBiz etti jedik.\nQyzymdy k\xc3\xb3rdi\xc5\x84iz be?\nQ'
Target: b'itaptardy taby\xc5\x84yz.\nQazir siz oqy\xc5\x84yz.\nShuly\xc7\xb5y\xc5\x84yzdy sheshi\xc5\x84iz.\nBiz etti jedik.\nQyzymdy k\xc3\xb3rdi\xc5\x84iz be?\nQy'


Create training batches
You used tf.data to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [29]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Build The Model
This section defines the model as a keras.Model subclass (For details see Making new Layers and Models via subclassing).

This model has three layers:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map each character-ID to a vector with embedding_dim dimensions;
tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
tf.keras.layers.Dense: The output layer, with vocab_size outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [30]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [34]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        if states is None:
            states = self.gru.get_initial_state(batch_size=tf.shape(inputs)[0], dtype=tf.float32)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [32]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](images/text_generation_training.png)

Note: For training you could use a keras.Sequential model here. To generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the Keras RNN guide.

Try the model
Now run the model to see that it behaves as expected.

First check the shape of the output:

In [1]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(MyModel)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

NameError: name 'dataset' is not defined

In [1]:
pip install boto3

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.6 MB 3.7 MB/s eta 0:00:04
   ------- -------------------------------- 2.4/12.6 MB 5.8 MB/s eta 0:00:02
   --------------- ------------------------ 5.0/12.6 MB 8.2 MB/s eta 0:00:01
   ------------------------ --------------- 7.6/12.6 MB 9.4 MB/s eta 0:00:01
   -------------------------------- ------- 10.2/12.6 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------  12.3/12.6 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 10.1 MB/s eta 0:00:00
